<a href="https://colab.research.google.com/github/CodeCraftrRyan/rlhaviland/blob/main/Lookbook_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, request, send_file
!pip install matplotlib-venn
!pip install reportlab
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Image, Paragraph, Spacer, Frame, KeepInFrame
from reportlab.lib import colors
from reportlab.lib.colors import Color
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import pandas as pd
from PIL import Image as PILImage
import os
from google.colab import drive
drive.mount('/content/drive')

from datetime import datetime

!pip install pillow # Make sure Pillow is installed for image processing
from PIL import Image as PILImage # Import the Image class from Pillow


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def add_logo(canvas, doc):
    """
    Adds a logo to the top right corner of the PDF.
    """
    logo_path = "/content/drive/My Drive/lookbook/EDFlogo_Square.jpg"
    logo.drawOn(canvas, letter[0] - logo.drawWidth - 0.5*inch, letter[1] - logo.drawHeight - 0.5*inch)

def add_timestamp(canvas, doc):
       """Adds a timestamp to the lower right corner of the PDF."""
       current_datetime = datetime.now().strftime("%Y-%m-%d %I:%M:%S %p")  # Changed format string
       timestamp = Paragraph(f"Generated on: {current_datetime}", styles["Normal"])
       timestamp.wrapOn(canvas, doc.width - 0.5*inch, 0.5*inch)
       timestamp.drawOn(canvas, doc.width - timestamp.width - 0.5*inch, 0.5*inch)

In [ ]:
# Load CSV data
df = pd.read_csv('/content/drive/My Drive/lookbook/lookbookexample.csv')

df.fillna('', inplace=True)  # Replaces all NaN values with an empty string

# Set up the PDF
pdf_filename = "lookbook.pdf"
doc = SimpleDocTemplate(pdf_filename, pagesize=letter, leftMargin=0.5 * inch, rightMargin=0.5 * inch, topMargin=0.5 * inch, bottomMargin=0.5 * inch,)
doc.build(elements, onFirstPage=add_logo, onLaterPages=add_logo)
elements = []

In [ ]:
# Define styles

custom_title_color = colors.Color(28/255, 68/255, 108/255)
custom_boarder_color = colors.Color(203/255, 212/255, 217/255)

styles = getSampleStyleSheet()
title_style = ParagraphStyle(
    "TitleStyle",
    parent=styles["Title"],
    fontName="Times-Roman",
    fontSize=20,
    textColor=custom_title_color,
    spaceAfter=10,
    spaceBefore=10,
    alignment=1,  # Centered
)
name_style = ParagraphStyle(
    "NameStyle",
    parent=styles["BodyText"],
    fontName="Times-Roman",
    fontSize=11,
    textColor=colors.black,
    spaceAfter=5,
    leading=18,
)
desc_style = ParagraphStyle(
    "DescStyle",
    parent=styles["BodyText"],
    fontName="Times-Roman",
    fontSize=10,
    textColor=colors.black,
    spaceAfter=5,
    leading=14,
    wordWrap="LTR",
)

In [ ]:
# Loop through each row in the CSV
for index, row in df.iterrows():
    name = row["Name"]
    title = row["Title"]
    company = row["Company"]
    additional = row["Additional"]
    description = row["Description"]
    image_path = row["Headshot"]  # Ensure image file exists

In [ ]:
# Title Page
logo = Image("/content/drive/My Drive/lookbook/EDFlogo_Square.jpg", width=0.5*inch, height=0.5*inch)
title = Paragraph("Elizabeth Dole Foundation Look-Book", title_style)
elements.append(title)
elements.append(Spacer(1, 10))


# Loop through each row in the CSV
for index, row in df.iterrows():
    name = row["Name"]
    title = row["Title"]
    company = row["Company"]
    additional = row["Additional"]
    description = row["Description"]
    image_path = row["Headshot"]

    # Load and resize the image
    try:
        img = PILImage.open(image_path)
        img = img.resize((75, 75), resample=PILImage.LANCZOS)
        temp_path = f"processed_images/temp_{index}.jpg"
        img.save(temp_path, quality=95)
        headshot = Image(temp_path)
    except:
        headshot = None  # If image fails to load, continue

    try:
        headshot = Image(image_path, width=1*inch, height=1*inch) # adjust dimensions as needed
    except:
        headshot = None  # If image fails to load, continue

    # Format text
    person_info = f"<b>{name}</b><br/>{company}, {title}<br/><i>{additional}</i><br/>"
    paragraph_name = Paragraph(person_info, name_style)
    description_info = f"{description}"
    paragraph_description= Paragraph(description_info, desc_style)

    # Create layout
    data = [
        [headshot, paragraph_name],
        [paragraph_description, ""]
    ]

    table = Table(data, colWidths=[90, 450], rowHeights=None)
    table.setStyle(TableStyle([
        ("VALIGN", (0, 0), (-1, -1), "TOP"),
        ("LEFTPADDING", (0, 0), (-1, -1), 5),
        ("RIGHTPADDING", (0, 0), (-1, -1), 5),
        ("BOX", (0, 0), (-1, -1), 2, custom_boarder_color),
        ("BACKGROUND", (0, 0), (-1, 0), colors.white),
        ("FONTNAME", (0, 0), (-1, 0), "Times-Roman"),
        ("BOTTOMPADDING", (0, 0), (-1, 0), 1),
        ("TOPPADDING", (0, 1), (-1, -1), .5),
        ("SPAN", (0,1), (1, 1)),
    ]))

    # Add to elements
    elements.append(table) # Appending table inside the loop
    elements.append(Spacer(1, 10))  # Space between entries

# Build PDF
doc.build(elements)

pdf_path = "/content/drive/My Drive/Lookbook/generated_pdf.pdf"

print(f"Lookbook PDF generated: {pdf_filename}")

Lookbook PDF generated: lookbook.pdf
